## Experimenting with Column Selection and Question Generation

This notebook aims to assess and improve the performance of the language model (LLM) by performing the following steps:

1. **Column Selection:**
   - **Objective:** Ask the LLM to identify two relevant columns from a schema that complement a given selected column.
   - **Process:** For each selected column, the LLM generates two additional column names from the list that are relevant for creating meaningful queries.

2. **Question Generation:**
   - **Objective:** Use the selected column pairs to formulate questions.
   - **Process:** For each pair of columns generated, the base model is tasked with generating two distinct questions about each column pair.

3. **SQL Query Generation:**
   - **Objective:** Evaluate how well the tuned model can generate SQL queries based on NLP questions.
   - **Process:** Take the NLP-generated questions and prompt the tuned model to produce corresponding SQL queries.

4. **Model Efficiency Evaluation:**
   - **Objective:** Assess the efficiency and accuracy of the model in generating SQL queries.
   - **Process:** Compare the SQL queries generated by the tuned model against expected results to evaluate performance.

5. **Training Data Enhancement:**
   - **Objective:** Determine if incorporating this type of data improves model generalization and performance.
   - **Process:** Integrate the newly generated questions and SQL pairs into the training dataset for further fine-tuning and generalization.

By experimenting with column selection, question generation, and SQL query creation, this approach aims to refine the model's ability to generate accurate SQL queries and enhance its performance over time. This iterative process helps in fine-tuning the model and ensuring better handling of diverse questions.


In [2]:
import lamini
import jsonlines
import sqlite3
import pandas as pd
from lamini import Lamini
import subprocess
from tqdm import tqdm
import json

In [5]:
# meta-llama/Meta-Llama-3-8B-Instruct
# 500ce81610ae6d4d45f84f6cd0936b79a306cd59e7aa549d3d14ecd45e8e969d
# RLHF : ca4bcce976e4d57620bf70d6045f04e8468b11c332c14128e42a3055812534dc
lamini.api_key = "1ac6f38b3e7f498fa2c3ed6ea9df68b9"
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

In [6]:
print(llm.generate("How are you?", max_new_tokens = 30))

 I hope you're doing well. I just wanted to check in and see if you're still interested in the job at [Company Name]. I know


In [7]:
def get_schema():
    return """\
0|Id|INTEGER eg. 1
1|fare_amount|REAL eg. 5.3
2|pickup_datetime|DATETIME eg. "2009-06-26 08:22:21"
3|passenger_count|INTEGER eg. 3
4|distance_miles|REAL eg. 12.5
5|pickup_statecode|TEXT eg. "NY"
6|pickup_statename|TEXT eg. "New York"
7|pickup_city|TEXT eg. "New York"
8|dropoff_statecode|TEXT eg. "NY"
9|dropoff_statename|TEXT eg. "New York"
10|dropoff_city|TEXT eg. "New York"
11|pickup_county|TEXT eg. "Kings"
12|dropoff_county|TEXT eg. "Kings"
"""

In [8]:
def make_llama_3_prompt(user, system=""):
    system_prompt = ""
    if system != "":
        system_prompt = (
            f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
        )
    return f"<|begin_of_text|>{system_prompt}<|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    

In [10]:
columns_info = [
    "Id|INTEGER eg. 1",
    "fare_amount|REAL eg. 5.3",
    "pickup_datetime|DATETIME eg. '2009-06-26 08:22:21'",
    "passenger_count|INTEGER eg. 3",
    "distance_miles|REAL eg. 12.5",
    "pickup_statecode|TEXT eg. 'NY'",
    "pickup_statename|TEXT eg. 'New York'",
    "pickup_city|TEXT eg. 'New York'",
    "dropoff_statecode|TEXT eg. 'NY'",
    "dropoff_statename|TEXT eg. 'New York'",
    "dropoff_city|TEXT eg. 'New York'",
    "pickup_county|TEXT eg. 'Kings'",
    "dropoff_county|TEXT eg. 'Kings'"
]

In [13]:
def make_columnpick_prompt(column_name):
    system = "You are an expert SQL query generator. You are provided with a database schema and a user's question about the data. Your task is to generate an accurate SQL query based on the schema to answer the user's question.\n"
    system += "Understand the following schema, as the insights derived from it will aid in making business decisions.\n"
    system += get_schema() + "\n"
    question = f"Given this column: {column_name}, provide two additional columns from the schema that can be used together to query and infer better results.\n"
    question += "Also, provide an explanation of your explanation for why the selected columns are best relevant to the given column.\n"
    question += "Output is a valid jsonl like {'explanation' : 'str', 'column1': 'str', 'column2': 'str'}\n"
    return make_llama_3_prompt(question, system)

In [14]:
def make_questiongen_prompt(column1, column2):
    system = "You are an expert SQL query generator. You are provided with a database schema and a user's question about the data. Your task is to generate an accurate SQL query based on the schema to answer the user's question.\n"
    system += "Understand the following schema, as the insights derived from it will aid in making business decisions.\n"
    system += get_schema() + "\n"
    question = f"Given these two columns , column1: {column1}, column2: {column2}\n"
    question += "Write two question which can be asked about this two columns, the answer to these questions should be a number, place or specific to some distribution.\n"
    question += "Output should be a valid jsonl like {'explanation' : 'str', 'column1': 'str', 'column2': 'str'}\n"
    question += "Output is a valid jsonl like {'question1' : 'str', 'question2': 'str'}.\n"
    return make_llama_3_prompt(question, system)

In [10]:
with jsonlines.open("sample_questions.jsonl", 'w') as writer:
    for index, column in enumerate(tqdm(columns_info, desc="Processing Columns")):
        prompt1 = make_columnpick_prompt(column)
        response1 = llm.generate(prompt1, output_type={
            "explanation": "str",
            "column1": "str",
            "column2": "str"
        })
        
        for i in range(2):
            column_name = "column" + str(i + 1)
            prompt2 = make_questiongen_prompt(column, response1[column_name])
            response2 = llm.generate(prompt2, output_type={
                "question1": "str",
                "question2": "str"
            })
            if (response2["question1"]).strip()[-1] != '?':
                response2["question1"] += '?'
            obj = {
                "question": response2["question1"]
            }
            if (response2["question2"]).strip()[-1] != '?':
                response2["question2"] += '?'
            writer.write(obj)
            obj["question"] = response2["question2"]
            writer.write(obj)

Processing Columns:   0%|          | 0/13 [00:00<?, ?it/s]DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/11" 200 None
Processing Columns:   8%|▊         | 1/13 [00:11<02:15, 11.33s/it]DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/11" 

In [11]:
llm = lamini.Lamini(model_name="ca4bcce976e4d57620bf70d6045f04e8468b11c332c14128e42a3055812534dc")

INFO:lamini.api.lamini:Using 3.10 InferenceQueue Interface


In [12]:
print(llm.generate("How are you?", max_new_tokens = 30))

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/11" 200 None


 I hope you're doing well. I just wanted to check in with you and see if everything is okay. You know I'm always here for you


In [33]:
    def make_prompt(question):
        system = "You are an expert SQL query generator. You are provided with a database schema and a user's question about the data. Your task is to generate an accurate SQL query based on the schema to answer the user's question.\n"
        system += "Understand the datatypes of the columns and try to generate a valid SQL query.\n"
        system += "Consider the following database table with the following schema and understand the column data types:\n"
        system += get_schema() + "\n"
        system += "Let's break down the question and think step-by-step to generate the SQL query.\n"
        system += "1. Understand the question.\n"
        system += "2. Identify the relevant columns from the schema.\n"
        system += "3. Formulate the conditions and relationships between columns.\n"
        system += "4. Construct the SQL query based on the identified columns and conditions.\n"
        system += "Ensure the SQL query is valid and ends with a semicolon."
        system += "Ensure the output is a valid json like {'sql':'str'} and the sql ends with a ;."
        return make_llama_3_prompt(question, system)

In [ ]:
with jsonlines.open("Data/sample_questions.jsonl", 'r') as reader, jsonlines.open("Data/generated_queries.jsonl", 'w') as writer:
     for q in tqdm(reader, desc="Generating SQL queries"):
         response = llm.generate(make_prompt(q["question"]), output_type={
            "sql": "str"
         })
         obj = {
            "question": q["question"],
            "sql": response["sql"]
         }
         writer.write(obj)

Generating SQL queries: 0it [00:00, ?it/s]DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/11" 200 None
Generating SQL queries: 1it [00:00,  1.96it/s]DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/11" 200 None
Generating SQL queries: 2it [00:01,  1.84it/s]DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/11" 200 None
Generating SQL queries: 3it [00:01,  1.95it/s]DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/11" 200 None
Generating SQL queries: 4it [00:02,  2.01it/s]DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ap

In [47]:
#cleaning data
# read the generated queries
# is if they are valid by executing the sql
# if the sql is valid append it to the filtered file
engine = sqlite3.connect("uber_rides.db")
cursor = engine.cursor()

with jsonlines.open("Data/generated_queries.jsonl", 'r') as reader:
    with jsonlines.open("Data/generated_queries_filtered.jsonl", 'w') as writer:
        for obj in reader:
            sql = obj["sql"]
            if sql.strip()[-1] != ';':
                sql += ';'
                obj["sql"] = sql
            df = pd.read_sql(sql, con=engine)
            if not df.empty:
                writer.write(obj)
